In [31]:
import requests
import json
from IPython.display import display
from bis2 import gc2
from bis import itis

### GAP Species with no TSN
The following report shows that do not currently have an ITIS TSN assigned. At the time these species were identified for modeling, ITIS may have contained no information, but there now may be information in ITIS that could be used. The link below the listing can be used to retrieve information from the ITIS web service.

This list can be reviewed to determine whether or not it is appropriate to go ahead and include a TSN in the metadata for these GAP species, setting up the information for further examination in the TIR.

In [33]:
r_withoutTSN = requests.get(gc2.sqlAPI("DataDistillery","BCB")+"&q=SELECT registration FROM tir.tir WHERE registration->>'source' = 'GAP Species' AND itis->>'MatchMethod' = 'Not Matched'").json()

print ("Number of GAP species with no ITIS designation: ",len(r_withoutTSN["features"]))

for feature in r_withoutTSN["features"]:
    registration = json.loads(feature["properties"]["registration"])
    _gapSpeciesCode = registration["identifiers"][next(index for (index, d) in enumerate(registration["identifiers"]) if d["type"] == "GAP_SpeciesCode")]["key"]
    _scientificname = registration["scientificname"]
    print (_scientificname, " - ", registration["commonname"], "(", gapSpeciesCode, ")")
    print (itis.getITISSearchURL(_scientificname,False))


Number of GAP species with no ITIS designation:  25
Tympanuchus phasianellus campestris  -  Prarie Sharp-tailed Grouse ( aSENEx )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Tympanuchus\%20phasianellus\%20campestris
Aplodontia rufa olympica  -  Mountain Beaver olympica ( aSENEx )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Aplodontia\%20rufa\%20olympica
Aplodontia rufa rufa  -  Mountain Beaver rufa ( aSENEx )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Aplodontia\%20rufa\%20rufa
Tamias minimus atristriatus  -  Penasco Chipmunk ( aSENEx )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Tamias\%20minimus\%20atristriatus
Oryzomys palustris natator  -  Key Oryzomys ( aSENEx )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Oryzomys\%20palustris\%20natator
Tamias minimus arizonensis  -  White Mountains Chipmunk ( aSENEx )
http://services.itis.gov/?wt=json&rows=10&q=nameWOInd:Tamias\%20minimus\%20arizonensis
Eleutherodactylus guttilatus  -  Spotted C

### Invalid TSNs
The following report shows GAP species where the assigned TSN is currently showing as invalid from ITIS. The information displayed includes a link to the invalid TSN from the ITIS web service, the reason the name is invalid, and a link to the ITIS service for the accepted TSN.

This may reflect a change in the taxonomy since the GAP species was identified and set up for modeling. These records can be reviewed to determine if some action is appropriate in the GAP species metadata.

In [42]:
r_invalidTSN = requests.get(gc2.sqlAPI("DataDistillery","BCB")+"&q=SELECT registration,itis FROM tir.tir WHERE registration->>'source' = 'GAP Species' AND itis->>'usage' <> 'valid'").json()

print ("Number of GAP species with invalid ITIS TSN: ",len(r_invalidTSN["features"]))
print ("------------")

for feature in r_invalidTSN["features"]:
    _registration = json.loads(feature["properties"]["registration"])
    _itis = json.loads(feature["properties"]["itis"])
    _gapSpeciesCode = _registration["identifiers"][next(index for (index, d) in enumerate(_registration["identifiers"]) if d["type"] == "GAP_SpeciesCode")]["key"]
    print (_registration["scientificname"], " - ", _registration["commonname"], "(", gapSpeciesCode, ")")
    print ("Assigned (invalid) TSN: ",itis.getITISSearchURL(_itis["tsn"],False))
    print ("Unaccept Reason: ",_itis["unacceptReason"])
    print ("Accepted TSN: ",itis.getITISSearchURL(_itis["acceptedTSN"][0],False))
    print ("------------")


Number of GAP species with invalid ITIS TSN:  128
------------
Spermophilus brunneus brunneus  -  Northern Idaho Ground Squirrel ( aSENEx )
Assigned (invalid) TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:180150
Unaccept Reason:  subsequent name/combination
Accepted TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:930316
------------
Dendroica nigrescens  -  Black-throated Gray Warbler ( aSENEx )
Assigned (invalid) TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:178896
Unaccept Reason:  subsequent name/combination
Accepted TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:950057
------------
Pipistrellus subflavus  -  Eastern Pipistrelle ( aSENEx )
Assigned (invalid) TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:180025
Unaccept Reason:  subsequent name/combination
Accepted TSN:  http://services.itis.gov/?wt=json&rows=10&q=tsn:948895
------------
Oporornis tolmiei  -  Macgillivray's Warbler ( aSENEx )
Assigned (invalid) TSN:  http://services.itis.gov/?wt=j